In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [24]:
data = pd.read_csv("/kaggle/input/factors-affecting-campus-placement/Placement_Data_Full_Class.csv")
#Remove Serial Number
data.drop("sl_no", axis=1, inplace=True)
data.info()

In [25]:
from sklearn import preprocessing
le=preprocessing.LabelEncoder()
data["gender"]=le.fit_transform(data["gender"])
data["hsc_s"]=le.fit_transform(data["hsc_s"])
data["degree_t"]=le.fit_transform(data["degree_t"])
data["workex"]=le.fit_transform(data["workex"])
data["status"]=le.fit_transform(data["status"])
data["specialisation"]=le.fit_transform(data["specialisation"])
data["ssc_b"]=le.fit_transform(data["ssc_b"])
data["hsc_b"]=le.fit_transform(data["hsc_b"])
data.info()


In [26]:
data.isnull().sum()

In [27]:
plt.figure(figsize=(12,10))
sns.heatmap(data.corr(),annot=True,cmap='Blues')

In [28]:
#board of education is not considered in placements so remove from df
data.drop(['ssc_b','hsc_b'],axis=1,inplace=True)

In [29]:
data.head()

In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
X = data[['gender', 'ssc_p', 'hsc_p', 'hsc_s', 'degree_p', 'degree_t', 'workex','etest_p', 'specialisation', 'mba_p',]]
y = data['status']

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [32]:
from sklearn.model_selection import GridSearchCV 
rf=RandomForestClassifier() 
param={'n_estimators':[100,200,350,500],'min_samples_leaf':[2,10,30]} 
grid=GridSearchCV(rf,param_grid=param,cv=5) 
grid.fit(X_train,y_train) 
print(grid.best_params_)

In [33]:
random_forest = RandomForestClassifier(n_estimators=350, min_samples_leaf=2, random_state=42)
random_forest.fit(X_train, y_train)
y_pred = random_forest.predict(X_test)
accuracy_score(y_test,y_pred)

In [34]:
print(classification_report(y_test,y_pred))